In [1]:
import pandas as pd

In [2]:
# Data
games_tourney = pd.read_csv('games_tourney_extended.csv')  # games_tourney.csv + games_season.csv three-input model prediction

from sklearn.model_selection import train_test_split

games_tourney_train, games_tourney_test = train_test_split(games_tourney, test_size=804)

In [3]:
games_tourney.head()

,Unnamed: 0,season,team_1,team_2,home,seed_diff,score_diff,score_1,score_2,won,pred
0,0,1985,288,73,0,-3,-9,41,50,0,0.128158
1,1,1985,5929,73,0,4,6,61,55,1,0.147784
2,2,1985,9884,73,0,5,-4,59,63,0,0.129091
3,3,1985,73,288,0,3,9,50,41,1,0.173936
4,4,1985,3920,410,0,1,-9,54,63,0,0.160824


In [4]:
# Simple two-output model

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Define the input
input_tensor = Input((2,))

# Define the output
output_tensor = Dense(2)(input_tensor)

# Define the model
model = Model(inputs=input_tensor, outputs=output_tensor)

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')

2022-02-23 00:14:01.254162: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-23 00:14:01.254268: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-23 00:14:06.644723: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-23 00:14:06.644816: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-23 00:14:06.644853: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (DESKTOP-FC4EGDV): /proc/driver/nvidia/version does not exist
2022-02-23 00:14:06.645193: I tensorflow/core/platform/cpu_fe

In [5]:
# Fit the model
model.fit(
    games_tourney_train[['seed_diff', 'pred']],
    games_tourney_train[['score_1', 'score_2']],
    verbose=2,
    epochs=100,
    batch_size=2**14
)

2022-02-23 00:14:06.942141: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100
1/1 - 1s - loss: 71.0904
Epoch 2/100
1/1 - 0s - loss: 71.0893
Epoch 3/100
1/1 - 0s - loss: 71.0881
Epoch 4/100
1/1 - 0s - loss: 71.0869
Epoch 5/100
1/1 - 0s - loss: 71.0857
Epoch 6/100
1/1 - 0s - loss: 71.0845
Epoch 7/100
1/1 - 0s - loss: 71.0833
Epoch 8/100
1/1 - 0s - loss: 71.0822
Epoch 9/100
1/1 - 0s - loss: 71.0810
Epoch 10/100
1/1 - 0s - loss: 71.0798
Epoch 11/100
1/1 - 0s - loss: 71.0786
Epoch 12/100
1/1 - 0s - loss: 71.0774
Epoch 13/100
1/1 - 0s - loss: 71.0763
Epoch 14/100
1/1 - 0s - loss: 71.0751
Epoch 15/100
1/1 - 0s - loss: 71.0739
Epoch 16/100
1/1 - 0s - loss: 71.0727
Epoch 17/100
1/1 - 0s - loss: 71.0715
Epoch 18/100
1/1 - 0s - loss: 71.0704
Epoch 19/100
1/1 - 0s - loss: 71.0692
Epoch 20/100
1/1 - 0s - loss: 71.0680
Epoch 21/100
1/1 - 0s - loss: 71.0668
Epoch 22/100
1/1 - 0s - loss: 71.0656
Epoch 23/100
1/1 - 0s - loss: 71.0644
Epoch 24/100
1/1 - 0s - loss: 71.0633
Epoch 25/100
1/1 - 0s - loss: 71.0621
Epoch 26/100
1/1 - 0s - loss: 71.0609
Epoch 27/100
1/1 - 0s

In [6]:
# Print the model's weights
print(model.get_weights())

# Print the column means of the training data
print(games_tourney_train.mean())

[array([[-0.02649776, -0.71656966],
       [ 0.28371283,  0.5153422 ]], dtype=float32), array([0.09999995, 0.09999995], dtype=float32)]
Unnamed: 0    2115.222157
season        2001.181050
team_1        5566.316910
team_2        5614.402332
home             0.000000
seed_diff        0.032070
score_diff       0.156560
score_1         71.198251
score_2         71.041691
won              0.506414
pred             0.149096
dtype: float64


In [7]:
# Evaluate the model on the tournament test data
print(model.evaluate(
    games_tourney_test[['seed_diff', 'pred']],
    games_tourney_test[['score_1', 'score_2']],
    verbose=0
))

70.96971130371094


Simple model for classifivation **and** regression

This kind of model is only possible with a neural network.

In [8]:
# Build a simple regressor/classifier model
from keras.models import Model
from keras.layers import Input, Dense

input_tensor = Input(shape=(1,))
output_tensor_reg = Dense(1)(input_tensor)
output_tensor_class = Dense(1, activation='sigmoid')(output_tensor_reg)

model = Model(input_tensor, [output_tensor_reg, output_tensor_class])

model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer='adam')

In [9]:
# Fit the combination classifier/regressor
X = games_tourney_test[['seed_diff']]
y_reg = games_tourney_test[['score_diff']]  # Regression target
y_class = games_tourney_test[['won']]  # Classification target
model.fit(X, [y_reg, y_class], epochs=100, verbose=2)

Epoch 1/100
26/26 - 1s - loss: 18.7772 - dense_1_loss: 18.2157 - dense_2_loss: 0.5614
Epoch 2/100
26/26 - 0s - loss: 18.6491 - dense_1_loss: 18.0882 - dense_2_loss: 0.5609
Epoch 3/100
26/26 - 0s - loss: 18.5224 - dense_1_loss: 17.9617 - dense_2_loss: 0.5608
Epoch 4/100
26/26 - 0s - loss: 18.3953 - dense_1_loss: 17.8348 - dense_2_loss: 0.5606
Epoch 5/100
26/26 - 0s - loss: 18.2671 - dense_1_loss: 17.7069 - dense_2_loss: 0.5603
Epoch 6/100
26/26 - 0s - loss: 18.1411 - dense_1_loss: 17.5810 - dense_2_loss: 0.5601
Epoch 7/100
26/26 - 0s - loss: 18.0129 - dense_1_loss: 17.4527 - dense_2_loss: 0.5602
Epoch 8/100
26/26 - 0s - loss: 17.8840 - dense_1_loss: 17.3241 - dense_2_loss: 0.5599
Epoch 9/100
26/26 - 0s - loss: 17.7571 - dense_1_loss: 17.1971 - dense_2_loss: 0.5599
Epoch 10/100
26/26 - 0s - loss: 17.6281 - dense_1_loss: 17.0684 - dense_2_loss: 0.5598
Epoch 11/100
26/26 - 0s - loss: 17.5054 - dense_1_loss: 16.9453 - dense_2_loss: 0.5601
Epoch 12/100
26/26 - 0s - loss: 17.3849 - dense_1_lo

In [10]:
model.get_weights()

[array([[0.6587068]], dtype=float32),
 array([-0.46156925], dtype=float32),
 array([[0.27043036]], dtype=float32),
 array([-0.07496227], dtype=float32)]

In [11]:
# Explore that part of the model that handles classification
from scipy.special import expit as sigmoid
score_diff = 1
weight = model.get_weights()[2][0, 0]
bias = model.get_weights()[3][0]

win_probability = sigmoid(score_diff * weight + bias)

win_probability

0.5487120212840203

In [12]:
# Evaluate the model on new data
X = games_tourney_test[['seed_diff']]
y_reg = games_tourney_test[['score_diff']]
y_class = games_tourney_test[['won']]

model.evaluate(X, [y_reg, y_class])

26/26 [==============================] - 0s 7ms/step - loss: 10.2521 - dense_1_loss: 9.6920 - dense_2_loss: 0.5601


[10.252132415771484, 9.692021369934082, 0.5601134300231934]

In [13]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

# Create an input layer with 2 columns
input_tensor = Input(shape=(2,))

# Create the first output
output_tensor_1 = Dense(1, activation='linear')(input_tensor)

# Create the second output (use the first output as input here)
output_tensor_2 = Dense(1, activation='sigmoid')(output_tensor_1)

# Create a model with two outputs
model = Model(input_tensor, [output_tensor_1, output_tensor_2])

In [14]:
# Compile and fit the model

# Import the Adam optimizer
from tensorflow.keras.optimizers import Adam

# Compile the model with 2 losses and the Adam optimizer with a higher learning rate
model.compile(loss=['mean_absolute_error', 'binary_crossentropy'], optimizer=Adam(learning_rate=.01))

# Fit the model to the tournament training data, with 2 inputs and 2 outputs
model.fit(
  games_tourney_train[['seed_diff', 'pred']],
  [
    games_tourney_train[['score_diff']],
    games_tourney_train[['won']]
  ],
  epochs=10,
  batch_size=16384
)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 11.4015 - dense_3_loss: 10.8273 - dense_4_loss: 0.5741
Epoch 2/10
1/1 [==============================] - 0s 5ms/step - loss: 11.3711 - dense_3_loss: 10.7931 - dense_4_loss: 0.5780
Epoch 3/10
1/1 [==============================] - 0s 6ms/step - loss: 11.3414 - dense_3_loss: 10.7592 - dense_4_loss: 0.5822
Epoch 4/10
1/1 [==============================] - 0s 7ms/step - loss: 11.3124 - dense_3_loss: 10.7259 - dense_4_loss: 0.5865
Epoch 5/10
1/1 [==============================] - 0s 6ms/step - loss: 11.2840 - dense_3_loss: 10.6931 - dense_4_loss: 0.5909
Epoch 6/10
1/1 [==============================] - 0s 7ms/step - loss: 11.2562 - dense_3_loss: 10.6607 - dense_4_loss: 0.5955
Epoch 7/10
1/1 [==============================] - 0s 7ms/step - loss: 11.2289 - dense_3_loss: 10.6288 - dense_4_loss: 0.6001
Epoch 8/10
1/1 [==============================] - 0s 8ms/step - loss: 11.2017 - dense_3_loss: 10.5971 - dense_4_loss: 0.6047
E

Inspect the model

In [15]:
# Print the model weights
model.get_weights()

# The last weight converts the predicted score difference
# to a predicted win probability

[array([[ 0.33478847],
        [-0.16776909]], dtype=float32),
 array([0.09144106], dtype=float32),
 array([[0.9834116]], dtype=float32),
 array([0.04443261], dtype=float32)]

In [18]:
# Print the training data means
games_tourney_train.mean()

Unnamed: 0    2115.222157
season        2001.181050
team_1        5566.316910
team_2        5614.402332
home             0.000000
seed_diff        0.032070
score_diff       0.156560
score_1         71.198251
score_2         71.041691
won              0.506414
pred             0.149096
dtype: float64

In [19]:
# Import the sigmoid function from scipy
from scipy.special import expit as sigmoid

# Weight from the model
weight = model.get_weights()[2][0, 0]

# Print the approximate win probability predicted close game
print(sigmoid(1 * weight))

# Print the approximate win probability predicted blowout game
print(sigmoid(10 * weight))

0.7277846296171921
0.9999464111584471


Evaluate on new data with two metrics

In [20]:
# Evaluate the model on new data
model.evaluate(
  games_tourney_test[['seed_diff', 'pred']],
  [
    games_tourney_test[['score_diff']],
    games_tourney_test[['won']]
  ],
  verbose=0
)

[11.178581237792969, 10.551734924316406, 0.6268458962440491]